In [1]:
!pip install panel plotly pandas numpy gspread

import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn
pn.extension("plotly")

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

sheet = gc.open("VENTAS_MES_12_INVENTARIOS_COMPLETO").worksheet("Sheet1")
data = sheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])

# Convertir valores numéricos
df["Ventas"] = pd.to_numeric(df["Ventas"], errors="coerce")
df["Precio"] = pd.to_numeric(df["Precio"], errors="coerce")

# VALORIZACIÓN ANUAL
df["Valor_anual"] = df["Ventas"] * df["Precio"]

# ORDENAR Y CALCULAR PORCENTAJES
df = df.sort_values(by="Valor_anual", ascending=False)
df["%"] = df["Valor_anual"] / df["Valor_anual"].sum()
df["%_acum"] = df["%"].cumsum()

# CLASIFICACIÓN ABC
df["ABC"] = np.where(df["%_acum"] <= 0.80, "A",
              np.where(df["%_acum"] <= 0.95, "B", "C"))

# -----------------------------
# POLÍTICAS DE INVENTARIO
# -----------------------------

# Parámetros generales
L_A = 2           # lead time A (días)
ss_A = 1.65       # factor Z
sigma_A = df["Ventas"].std()

L_B = 5           # lead time B
T_B = 5           # periodo revisión
ss_B = 1.30       # factor Z
sigma_B = df["Ventas"].std()

def politicas(row):
    if row["ABC"] == "A":
        # Revisión continua (Q)
        R = row["Ventas"] * L_A + ss_A * sigma_A
        return f"Política Q | Punto de Reorden R={R:.1f}"
    else:
        # Revisión periódica (P)
        S = row["Ventas"] * (L_B + T_B) + ss_B * sigma_B
        return f"Política P | Nivel máximo S={S:.1f}"

df["Política"] = df.apply(politicas, axis=1)

# GRÁFICOS
fig_abc = px.bar(df, x="Producto", y="Valor_anual",
                 color="ABC", title="Clasificación ABC")

fig_politicas = px.histogram(df, x="ABC", color="ABC",
                             title="Distribución de Políticas")

# DASHBOARD PANEL
dashboard = pn.Column(
    "# 📦 Dashboard Inventarios Distribuidora 2L",
    "## Clasificación ABC",
    fig_abc,
    "## Políticas de Inventario (Q y P)",
    fig_politicas,
    "## Tabla completa",
    pn.widgets.DataFrame(df, autosize_mode="fit_viewport")
)

dashboard.servable()
dashboard


/tmp/ipython-input-3251334079.py:7: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension("plotly")


KeyError: 'Ventas'